# Impact of Affordable Housing on Home Values and Houshold Income

This notebook uses a dataset that has been cleansed for analysis using the following conditions:
- Residential properties (Single-Family and Condo)
- Non-null census tracts
- Sales transactions above $1000
- Complete duplicates removed
- Sales processed after 2010

This dataset is pre-sorted, ascending, in the following order: _tract_, _apn_ and _saleamount_.<br><br>
Median income level data is obtained from the 'B19013' census table found in the census.sqlite file. Top and bottom quartiles of the census tracks, according to the median of the 'median household incomes' for tracts from 2010 to 2020, are identified as well the impact of affordable housing development status on median home values and median household incomes.

Housing data is obtained from a CSV file named 'sales_cleansed_data.csv' that is created by the notebook 'sales_cleansed_data.ipynb' that is in the same GitHub repository.

In [ ]:
import pandas as pd
import numpy as np
import sqlite3 as sql
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick
from matplotlib.ticker import StrMethodFormatter
%matplotlib inline

### Establish two arrays for tracts with and without AH (hard-copied from a previous analysis)

In [ ]:
# Tracts that had affordable housing developments before 2010
AHtracts = np.array([  '10301', '10302', '10401', '10402', '10501', '10702', '10903',
                       '11001', '11002', '11200', '11300', '11400', '11700', '11800',
                       '12100', '12600', '12701', '12702', '12801', '13300', '13500',
                       '13601', '13700', '13800', '13900', '14300', '14400', '15502',
                       '15610', '15618', '15620', '15622', '15623', '15628', '15629',
                       '15900', '16100', '16200', '17000', '17100', '17401', '18202',
                       '19005', '19105', '19116', '19200', '19300', '19400', '19500'])

# Tracts that DO NOT have affordable housing
REGtracts = np.array([ '10103', '10104', '10105', '10106', '10303', '10502', '10602',
                       '10701', '10801', '10802', '10901', '11100', '11500', '11600',
                       '12200', '12802', '13000', '13100', '13201', '13202', '13400',
                       '13602', '14200', '14800', '15100', '15200', '15300', '15401',
                       '15402', '15404', '15405', '15501', '15609', '15613', '15614',
                       '15617', '15619', '15625', '15626', '15627', '15630', '15631',
                       '15700', '15802', '15803', '16000', '16300', '16400', '16500',
                       '16600', '16700', '16800', '16900', '17300', '17402', '17701',
                       '17702', '17800', '17901', '17902', '18000', '18101', '18102',
                       '18201', '18203', '18301', '18302', '18401', '18404', '18405',
                       '18407', '18408', '18409', '18410', '18500', '18602', '18700',
                       '18801', '18803', '18804', '18901', '18902', '18904', '18905',
                       '19003', '19004', '19006', '19106', '19109', '19110', '19111',
                       '19112', '19114', '19115', '19117', '19118', '19600'])

#### Read in the Census data as a DataFrame using SQLite
This database contains data gathered from the US Census [American Community Survey](https://www.census.gov/data/developers/data-sets.html) for the years 2009 through 2020. 
- For the tables and variables included, see the data dictionary.
- Be aware that census tracts change around 
- Census tracts in the housing database PROBABLY come from the new census tract info (VERIFY THIS)
- Can use https://www.census.gov/geographies/mapping-files/time-series/geo/tiger-line-file.2020.html to compare older census property tract numbers to new ones
- Can probably filter down to 2010 to Present

In [ ]:
cens_conn = sql.connect('../data/census.sqlite')
cur = cens_conn.cursor()

query_init = "Select name FROM sqlite_master WHERE type='table' ORDER BY name;"
cur.execute(query_init)
census_tables = (cur.fetchall())

In [ ]:
print(census_tables)

#### Clean and Format the Census Data

In [ ]:
# Save the Median Household Income data from the Census dataset as a DataFrame
median_incomes = pd.read_sql("SELECT * FROM B19013", cens_conn)
median_incomes.head()

In [ ]:
median_incomes.info()

In [ ]:
# Convert tracts from integers to strings
median_incomes['tract'] = median_incomes.tract.astype('string')
median_incomes.info()

In [ ]:
# Filter for records between 2010 and 2020
median_incomes = median_incomes.loc[median_incomes.year.isin(range(2010, 2021))]

In [ ]:
# Remove records with a '-666666666' value
median_incomes = median_incomes.loc[median_incomes.value != -666666666]

In [ ]:
# Keep only the tracts with more than 10 years' of income information
income_counts = median_incomes.tract.value_counts().to_dict()
valid_tracts = [tract for tract in income_counts if income_counts[tract] >= 10]
median_incomes = median_incomes.loc[median_incomes.tract.isin(valid_tracts)]

In [ ]:
# Subset for the relevant fields
median_incomes = median_incomes[['year', 'tract', 'value']].sort_values(by=['year', 'tract', 'value']).reset_index(drop=True)
median_incomes = median_incomes.rename(columns={'value':'median_income'})

### Determine Bottom and Top 25% of Tracts, according to Median Household Income

#### Calculate the Median Across All Years (from 2010 to 2020) of That Tract's Collective 'Median Household Incomes'.

In [ ]:
income_per_tract = median_incomes.groupby('tract').median_income.median().sort_values().astype(int).reset_index()

#### For All Tracks

In [ ]:
bottom25_tracts = income_per_tract.head(round(len(income_per_tract)*0.25)).tract.to_list()
top25_tracts = income_per_tract.tail(round(len(income_per_tract)*0.25)).tract.to_list()

#### For AH Tracks

In [ ]:
bottom25_AHtracts = income_per_tract[income_per_tract.tract.isin(AHtracts)].head(round(len(income_per_tract)*0.25)).tract.to_list()
top25_AHtracts = income_per_tract[income_per_tract.tract.isin(AHtracts)].tail(round(len(income_per_tract)*0.25)).tract.to_list()

#### For Non-AH Tracks

In [ ]:
bottom25_REGtracts = income_per_tract[income_per_tract.tract.isin(REGtracts)].head(round(len(income_per_tract)*0.25)).tract.to_list()
top25_REGtracts = income_per_tract[income_per_tract.tract.isin(REGtracts)].tail(round(len(income_per_tract)*0.25)).tract.to_list()

income_per_REGtract.head(round(len(income_per_tract)*0.25)).tract.to_list()
                   .head(round(len(income_per_tract)*0.25)).tract.to_list() 

### Plot the Distribution of Median Household Incomes for AH vs. Non-AH tracts

#### Calculate the Median Income for Each AH Tract (Across All Years)

In [ ]:
income_per_AHtract = median_incomes[median_incomes.tract.isin(AHtracts)].groupby('tract').median_income.median().reset_index()
income_per_AHtract['median_income'] = income_per_AHtract.median_income.astype(int)
income_per_AHtract = income_per_AHtract.sort_values(by='median_income').reset_index(drop=True)

#### Calculate the Median Income for Each Non-AH Tract (Across All Years)

In [ ]:
# Determine the median income for each NON-AH tract (the median income across all years)
income_per_REGtract = median_incomes[median_incomes.tract.isin(REGtracts)].groupby('tract').median_income.median().reset_index()
income_per_REGtract['median_income'] = income_per_REGtract.median_income.astype(int)
income_per_REGtract = income_per_REGtract.sort_values(by='median_income').reset_index(drop=True)

#### Plot the Results

In [ ]:
fig, ax = plt.subplots(figsize=(10,6))

sns.histplot(x='median_income', data=income_per_REGtract, bins=15, stat='proportion', alpha=1, color='#0F0064')
sns.histplot(x='median_income', data=income_per_AHtract, bins=12, stat='proportion', alpha=0.7, color='#F3C400')

plt.title('Distribution of Median Household Income',
          fontsize=14,
          fontweight='bold')
plt.xlabel('Median Household Income',
           fontsize=12,
           fontweight='bold')
plt.ylabel('Percentage of Total Tracts',
           fontsize=12,
           fontweight='bold')

ax.legend(['Without AH', 'With AH\n(Developments Built Before 2010)'])
ax.xaxis.set_major_formatter(StrMethodFormatter('${x:,g}'))
ax.yaxis.set_major_formatter(mtick.PercentFormatter(xmax=1, decimals=0))

plt.tight_layout()
#plt.savefig('../images/dist_incomes.png')

## Calculate Median Incomes for Each Year and the Average Percent Income Growth<br>(2010 vs. 2020)

#### Conclusions:
     Lower Income Tracts:
     a.    Overall: 7% per year
     b. Without AH: 4% per year
     c.    With AH: 7% per year
     
     Upper Income Tracts:
     a.    Overall: 3% per year
     b. Without AH: 3% per year
     c.    With AH: 5% per year
     
     All Income Tracts:
     a.    Overall: 4% per year
     b. Without AH: 3% per year
     c.    With AH: 5% per year
     

### For All Tracts

#### All Incomes

In [ ]:
income_per_year = median_incomes.groupby('year').median_income.median().to_frame().reset_index()
income_per_year['median_income'] = income_per_year.median_income.round(-2).astype(int)
income_per_year = income_per_year.sort_values(by='year').reset_index(drop=True)

In [ ]:
# Percent growth per year
income2010 = income_per_year._get_value(0, 'median_income')
income2020 = income_per_year._get_value(10, 'median_income')

income_change = income2020 - income2010

perc_incr = int(round(income_change / income2010 * 100, -1))
perc_incr_yearly = int(round(perc_incr / 10))
print('Median Household Incomes for all tracts increased by', str(perc_incr) + '% or by an average of',
      str(perc_incr_yearly) + '% per year.')

#### Lower Incomes

In [ ]:
bottom25_income_per_year = median_incomes[median_incomes.tract.isin(bottom25_tracts)].groupby('year').median_income.median().to_frame().reset_index()
bottom25_income_per_year['median_income'] = bottom25_income_per_year.median_income.round(-2).astype(int)
bottom25_income_per_year = bottom25_income_per_year.sort_values(by='year').reset_index(drop=True)

In [ ]:
# Percent growth per year
bottom25_income2010 = bottom25_income_per_year._get_value(0, 'median_income')
bottom25_income2020 = bottom25_income_per_year._get_value(10, 'median_income')

bottom25_income_change = bottom25_income2020 - bottom25_income2010

bottom25_perc_incr = int(round(bottom25_income_change / bottom25_income2010 * 100, -1))
bottom25_perc_incr_yearly = int(round(bottom25_perc_incr / 10))
print('Median Household Incomes for all tracts in the Bottom 25% increased by', str(bottom25_perc_incr) + '% or by an average of',
      str(bottom25_perc_incr_yearly) + '% per year.')

#### Upper Incomes

In [ ]:
top25_income_per_year = median_incomes[median_incomes.tract.isin(top25_tracts)].groupby('year').median_income.median().to_frame().reset_index()
top25_income_per_year['median_income'] = top25_income_per_year.median_income.round(-2).astype(int)
top25_income_per_year = top25_income_per_year.sort_values(by='year').reset_index(drop=True)

In [ ]:
# Percent growth per year
top25_income2010 = top25_income_per_year._get_value(0, 'median_income')
top25_income2020 = top25_income_per_year._get_value(10, 'median_income')

top25_income_change = top25_income2020 - top25_income2010

top25_perc_incr = int(round(top25_income_change / top25_income2010 * 100, -1))
top25_perc_incr_yearly = int(round(top25_perc_incr / 10))
print('Median Household Incomes for all tracts in the Top 25% increased by', str(top25_perc_incr) + '% or by an average of',
      str(top25_perc_incr_yearly) + '% per year.')

### For AH Tracts

#### All Incomes

In [ ]:
AHincome_per_year = median_incomes[median_incomes.tract.isin(AHtracts)].groupby('year').median_income.median().to_frame().reset_index()
AHincome_per_year['median_income'] = AHincome_per_year.median_income.round(-2).astype(int)
AHincome_per_year = AHincome_per_year.sort_values(by='year').reset_index(drop=True)

In [ ]:
# Percent growth per year
AHincome2010 = AHincome_per_year._get_value(0, 'median_income')
AHincome2020 = AHincome_per_year._get_value(10, 'median_income')

AHincome_change = AHincome2020 - AHincome2010

AHperc_incr = int(round(AHincome_change / AHincome2010 * 100, -1))
AHperc_incr_yearly = int(round(AHperc_incr / 10))
print('Median Household Incomes for all AH tracts increased by', str(AHperc_incr) + '% or by an average of',
      str(AHperc_incr_yearly) + '% per year.')

#### Lower Incomes

In [ ]:
bottom25_AHincome_per_year = median_incomes[median_incomes.tract.isin(bottom25_AHtracts)].groupby('year').median_income.median().to_frame().reset_index()
bottom25_AHincome_per_year['median_income'] = bottom25_AHincome_per_year.median_income.round(-2).astype(int)
bottom25_AHincome_per_year = bottom25_AHincome_per_year.sort_values(by='year').reset_index(drop=True)

In [ ]:
# Percent growth per year
bottom25_AHincome2010 = bottom25_AHincome_per_year._get_value(0, 'median_income')
bottom25_AHincome2020 = bottom25_AHincome_per_year._get_value(10, 'median_income')

bottom25_AHincome_change = bottom25_AHincome2020 - bottom25_AHincome2010

bottom25_AHperc_incr = int(round(bottom25_AHincome_change / bottom25_AHincome2010 * 100, -1))
bottom25_AHperc_incr_yearly = int(round(bottom25_AHperc_incr / 10))
print('Median Household Incomes for AH tracts in the Bottom 25% increased by', str(bottom25_AHperc_incr) + '% or by an average of',
      str(bottom25_AHperc_incr_yearly) + '% per year.')

#### Upper Incomes

In [ ]:
top25_REGincome_per_year = median_incomes[median_incomes.tract.isin(top25_REGtracts)].groupby('year').median_income.median().to_frame().reset_index()
top25_REGincome_per_year['median_income'] = top25_REGincome_per_year.median_income.round(-2).astype(int)
top25_REGincome_per_year = top25_REGincome_per_year.sort_values(by='year').reset_index(drop=True)

In [ ]:
# Percent growth per year
top25_REGincome2010 = top25_REGincome_per_year._get_value(0, 'median_income')
top25_REGincome2020 = top25_REGincome_per_year._get_value(10, 'median_income')

top25_REGincome_change = top25_REGincome2020 - top25_REGincome2010

top25_REGperc_incr = int(round(top25_REGincome_change / top25_REGincome2010 * 100, -1))
top25_REGperc_incr_yearly = int(round(top25_REGperc_incr / 10))
print('Median Household Incomes for all tracts in the Top 25% increased by', str(top25_REGperc_incr) + '% or by an average of',
      str(top25_REGperc_incr_yearly) + '% per year.')

### Non-AH Tracts

#### All Incomes

In [ ]:
REGincome_per_year = median_incomes[median_incomes.tract.isin(REGtracts)].groupby('year').median_income.median().to_frame().reset_index()
REGincome_per_year['median_income'] = REGincome_per_year.median_income.round(-2).astype(int)
REGincome_per_year = REGincome_per_year.sort_values(by='year').reset_index(drop=True)

In [ ]:
# Percent growth per year
REGincome2010 = REGincome_per_year._get_value(0, 'median_income')
REGincome2020 = REGincome_per_year._get_value(10, 'median_income')

REGincome_change = REGincome2020 - REGincome2010

REGperc_incr = int(round(REGincome_change / REGincome2010 * 100, -1))
REGperc_incr_yearly = int(round(REGperc_incr / 10))
print('Median Household Incomes for REG tracts increased by', str(REGperc_incr) + '% or by an average of',
      str(REGperc_incr_yearly) + '% per year.')

#### Lower Incomes

In [ ]:
bottom25_REGincome_per_year = median_incomes[median_incomes.tract.isin(bottom25_REGtracts)].groupby('year').median_income.median().to_frame().reset_index()
bottom25_REGincome_per_year['median_income'] = bottom25_REGincome_per_year.median_income.round(-2).astype(int)
bottom25_REGincome_per_year = bottom25_REGincome_per_year.sort_values(by='year').reset_index(drop=True)

In [ ]:
# Percent growth per year
bottom25_REGincome2010 = bottom25_REGincome_per_year._get_value(0, 'median_income')
bottom25_REGincome2020 = bottom25_REGincome_per_year._get_value(10, 'median_income')

bottom25_REGincome_change = bottom25_REGincome2020 - bottom25_REGincome2010

bottom25_REGperc_incr = int(round(bottom25_REGincome_change / bottom25_REGincome2010 * 100, -1))
bottom25_REGperc_incr_yearly = int(round(bottom25_REGperc_incr / 10))
print('Median Household Incomes for Non-REG tracts in the Bottom 25% increased by', str(bottom25_REGperc_incr) + '% or by an average of',
      str(bottom25_REGperc_incr_yearly) + '% per year.')

#### Upper Incomes

In [ ]:
top25_AHincome_per_year = median_incomes[median_incomes.tract.isin(top25_AHtracts)].groupby('year').median_income.median().to_frame().reset_index()
top25_AHincome_per_year['median_income'] = top25_AHincome_per_year.median_income.round(-2).astype(int)
top25_AHincome_per_year = top25_AHincome_per_year.sort_values(by='year').reset_index(drop=True)

In [ ]:
# Percent growth per year
top25_AHincome2010 = top25_AHincome_per_year._get_value(0, 'median_income')
top25_AHincome2020 = top25_AHincome_per_year._get_value(10, 'median_income')

top25_AHincome_change = top25_AHincome2020 - top25_AHincome2010

top25_AHperc_incr = int(round(top25_AHincome_change / top25_AHincome2010 * 100, -1))
top25_AHperc_incr_yearly = int(round(top25_AHperc_incr / 10))
print('Median Household Incomes for Non-AH tracts in the Top 25% increased by', str(top25_AHperc_incr) + '% or by an average of',
      str(top25_AHperc_incr_yearly) + '% per year.')

## Analysis of Home Values

### Read in the Housing Sales Data
Housing data is obtained from a CSV file named 'sales_cleansed_data.csv' that is created by the notebook 'sales_cleansed_data.ipynb' that is in the same GitHub repository.

In [ ]:
sales_original_df = pd.read_csv('../data/sales_cleansed_data.csv', index_col=0, dtype={'tract':'string'})
sales_original_df.info()

In [ ]:
sales_df = sales_original_df.copy(deep=True)

#### Calculate Median Home Value Per APN Per Year

In [ ]:
apn_value_per_year = sales_df.groupby(['tract', 'apn', 'saleyear']).saleamount.median().round(-2).astype(int).reset_index()
apn_value_per_year = apn_value_per_year.rename(columns={'saleyear':'year', 'saleamount':'median_value'})

### Subset for Non-AH tracts in the bottom 25% median income bracket

#### For All Tracts

In [ ]:
bottom25_sales = apn_value_per_year.loc[apn_value_per_year.tract.isin(bottom25_tracts)]
top25_sales = apn_value_per_year.loc[apn_value_per_year.tract.isin(top25_tracts)]

#### For AH Tracts

In [ ]:
bottom25_AHtract_sales = apn_value_per_year.loc[apn_value_per_year.tract.isin(bottom25_AHtracts)]
top25_AHtract_sales = apn_value_per_year.loc[apn_value_per_year.tract.isin(top25_AHtracts)]

#### For Non-AH Tracts

In [ ]:
bottom25_REGtract_sales = apn_value_per_year.loc[apn_value_per_year.tract.isin(bottom25_REGtracts)]
top25_REGtract_sales = apn_value_per_year.loc[apn_value_per_year.tract.isin(top25_REGtracts)]

## Compare Median Home Prices for Bottom and Top 25% Median Income Level Tracts from 2010-2022

### Calculate Median Home Prices for Bottom and Top 25% Tracts WITH Affordable Housing

In [ ]:
# Calculate the median home price for each of the bottom 25% AH tracts
bottom25_AHtract_values = bottom25_AHtract_sales.groupby('year').median_value.median().round(-3).astype(int).reset_index().sort_values(by='year')

In [ ]:
# Calculate the median home price for each of the top 25% AH tracts
top25_AHtract_values = top25_AHtract_sales.groupby('year').median_value.median().round(-3).astype(int).reset_index().sort_values(by='year')

### Calculate Median Home Prices for Bottom and Top 25% Tracts WITHOUT Affordable Housing

In [ ]:
# Calculate the median home price for each year for the bottom 25% NON-AH tracts
bottom25_REGtract_values = bottom25_REGtract_sales.groupby('year').median_value.median().round(-3).astype(int).reset_index().sort_values(by='year')

In [ ]:
# Calculate the median home price for each of the top 25% AH tracts
top25_REGtract_values = top25_REGtract_sales.groupby('year').median_value.median().round(-3).astype(int).reset_index().sort_values(by='year')

## Try to Plot them Side by Side

In [ ]:
fig, axes = plt.subplots(figsize=(10,6), nrows=1, ncols=2)

axes[0].axvline(x=2020, color='#E0E0E0', linestyle=':', linewidth=1)
axes[1].axvline(x=2020, color='#E0E0E0', linestyle=':', linewidth=1)

ax1, = axes[0].plot('year', 'median_value', data=bottom25_REGtract_values, color='#0F0064')
ax2, = axes[0].plot('year', 'median_value', data=bottom25_AHtract_values, color='#F3C400')
axes[1].plot('year', 'median_value', data=top25_REGtract_values, color='#0F0064')
axes[1].plot('year', 'median_value', data=top25_AHtract_values, color='#F3C400')

axes[0].set_ylim([25000,525000])
axes[1].set_ylim([25000,525000])

axes[0].legend(handles=[ax1,
                        ax2],
               labels=['Without AH',
                       'With AH\n(Developments Built Before 2010)'],
               loc='upper left',
               fontsize=9)

plt.suptitle('Growth of Median Home Value' +
             '\n(Tracts in the Bottom and Top 25% of Median Household Incomes)',
                  fontsize=14,
                  fontweight='bold')

axes[0].set_title('Bottom 25%',
                  fontsize=12,
                  fontweight='bold')
axes[1].set_title('Top 25%',
                  fontsize=12,
                  fontweight='bold')

axes[1].set_xlabel('Year',
                   fontsize=12,
                   fontweight='bold',
                   loc='left')
axes[0].set_ylabel('Home Value',
                   fontsize=12,
                   fontweight='bold')

axes[0].yaxis.set_major_formatter(StrMethodFormatter('${x:,g}'))
axes[1].set_yticklabels([],
                   fontsize=12,
                   fontweight='bold')

axes[0].grid(visible=True, axis='y', color='#EEEEEE')
axes[1].grid(visible=True, axis='y', color='#EEEEEE')

plt.tight_layout()
#plt.savefig('../images/value_wvswo_topbot25.png')

### Percent Growth of Home Prices for Lower Incomes, 2010 vs 2020

#### For AH Tracts

In [ ]:
bottom25_AHvalue2010 = bottom25_AHtract_values._get_value(0, 'median_value')
bottom25_AHvalue2020 = bottom25_AHtract_values._get_value(10, 'median_value')

bottom25_AHvaldiff = bottom25_AHvalue2020 - bottom25_AHvalue2010

bottom25AH_percincr = int(round(bottom25_AHvaldiff / bottom25_AHvalue2010 * 100, -1))
bottom25AH_percincr_yearly = int(round((bottom25_AHvaldiff / bottom25_AHvalue2010 * 100) / 10))
print('Home prices for tracts WITH AH increased by', str(bottom25AH_percincr) + '% or by an average of',
      str(bottom25AH_percincr_yearly) + '% per year.')

#### For Non-AH Tracts

In [ ]:
bottom25_REGvalue2010 = bottom25_REGtract_values._get_value(0, 'median_value')
bottom25_REGvalue2020 = bottom25_REGtract_values._get_value(10, 'median_value')

bottom25_REGvaldiff = bottom25_REGvalue2020 - bottom25_REGvalue2010

bottom25REG_percincr = int(round(bottom25_REGvaldiff / bottom25_REGvalue2010 * 100, -1))
bottom25REG_percincr_yearly = int(round((bottom25_REGvaldiff / bottom25_REGvalue2010 * 100) / 10))
print('Home prices for tracts without AH increased by', str(bottom25REG_percincr) + '% or by an average of',
      str(bottom25REG_percincr_yearly) + '% per year.')

### Percent Growth of Home Prices for Higher Incomes, 2010 vs 2020

#### For AH Tracts

In [ ]:
top25_AHvalue2010 = top25_AHtract_values._get_value(0, 'median_value')
top25_AHvalue2020 = top25_AHtract_values._get_value(10, 'median_value')

top25_AHvaldiff = top25_AHvalue2020 - top25_AHvalue2010

top25AH_percincr = int(round(top25_AHvaldiff / top25_AHvalue2010 * 100, -1))
top25AH_percincr_yearly = int(round((top25_AHvaldiff / top25_AHvalue2010 * 100) / 10))
print('Home prices for tracts WITH AH increased by', str(top25AH_percincr) + '% or by an average of',
      str(top25AH_percincr_yearly) + '% per year.')

#### For Non-AH Tracts

In [ ]:
top25_REGvalue2010 = top25_REGtract_values._get_value(0, 'median_value')
top25_REGvalue2020 = top25_REGtract_values._get_value(10, 'median_value')

top25_REGvaldiff = top25_REGvalue2020 - top25_REGvalue2010

top25REG_percincr = int(round(top25_REGvaldiff / top25_REGvalue2010 * 100, -1))
top25REG_percincr_yearly = int(round((top25_REGvaldiff / top25_REGvalue2010 * 100) / 10))
print('Home prices for tracts without AH increased by', str(top25REG_percincr) + '% or by an average of',
      str(top25REG_percincr_yearly) + '% per year.')